In [ ]:
from IPython import get_ipython
from IPython.display import display
   # %%
   # -----------------------------
   # 🛠️ 1. Install Required Packages
   # -----------------------------
!pip install langchain langchain-openai langchain-community langchain-chroma python-dotenv
!pip install pypdf
   # -----------------------------
   # 🔐 2. Environment Setup
   # -----------------------------
import os
from dotenv import load_dotenv

   # Set OpenAI API Key directly here
os.environ["OPENAI_API_KEY"] = "sk--iIesrgwskbU4s1HxtelBn1sRt3bUvi9Tv2Dpu02M0T3BlbkFJ7aal4rANaVqWXRSI72klvx4Q4Zg7DiLslKikzi_8wA"
load_dotenv()

   # -----------------------------
   # 📥 3. Ingest Documents (PDFs must already be in /data)
   # -----------------------------
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from uuid import uuid4

DATA_PATH = "/content"
CHROMA_PATH = "chroma_db"

os.makedirs(DATA_PATH, exist_ok=True)

embedding_function = OpenAIEmbeddings()

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embedding_function,
    persist_directory=CHROMA_PATH,
   )

# Load and process PDFs from /data folder
loader = PyPDFDirectoryLoader(DATA_PATH)
raw_documents = loader.load()

   # Print the loaded documents to check if any were loaded
print(f"Loaded {len(raw_documents)} documents: {raw_documents}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(raw_documents)
uuids = [str(uuid4()) for _ in range(len(chunks))]

vector_store.add_documents(documents=chunks, ids=uuids)

   # -----------------------------
   # 💬 4. Start Chatbot Loop
   # -----------------------------
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.5, model='gpt-4o-mini')
retriever = vector_store.as_retriever(search_kwargs={'k': 5})

print("\n✅ Ingestion complete. Start chatting!")

history = []

while True:
    message = input("\n🧠 Ask a question (or type 'exit' to quit): ")
    if message.lower() == 'exit':
        print("👋 Exiting chatbot. Bye!")
        break

    docs = retriever.invoke(message)
    knowledge = "\n\n".join(doc.page_content for doc in docs)

    rag_prompt = f"""
       You are an ai assistant chatbot answering questions only using the provided knowledge to the users.If the user asks for an email address of anyone please give.
       Do not use any internal knowledge or say the source of your knowledge.

       Question: {message}
       Conversation history: {history}
       Knowledge: {knowledge}
       """

    response = ""
    for part in llm.stream(rag_prompt):
        response += part.content

    print("\n🤖 Answer:\n", response.strip())
    history.append((message, response.strip()))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.9 MB/s eta 0:00:00


KeyboardInterrupt: Interrupted by user